In [1]:
from bs4 import BeautifulSoup
import urllib
import re
import datetime
import pandas as pd
import numpy as np

In [2]:
def get_division(tags, did):
    
    # For a webpage corresponding to a division homepage, record division and competitor details 
    # This function returns two arguments: the division description (div_details), and the division competitors (team_table)
    # If the division is a team league, then the competitors are teams containing multiple players
    
    div_name = str(tags[4].text)
    location = str(tags[6].text)
    fin = str(tags[len(tags)-4].text)
    num_competitors = int(tags[len(tags)-4].text) if int(tags[10].text) and '-' not in fin and 'View' not in fin else 0
    form = str(tags[12].text)
    h2h_bonus = str(tags[14].text)
    oneten_rule = str(tags[18].text)
    forfeit_saviour = str(tags[22].text)
    dates = [str(i) for i in tags[28].text.split(':')[1:]]
    dates1 = [pd.to_datetime('-'.join(date.split(' ')[1:4])) for date in dates]
    weeks = len(dates1)
    div_details = [div_name, did, location, weeks, num_competitors, form, h2h_bonus, oneten_rule, forfeit_saviour, dates1]

    team_tags = [tags[32 + i*4] for i in range(0,num_competitors)]
    team_names = [str(tag.strong.text) for tag in team_tags]
    team_ids = [str(tag.find_all('a')[1]).split('&')[0][-5:] for tag in team_tags]
    team_scores = [int(str(tag.text.split(':')[-2]).split('BYE')[0]) if 'BYE' in tag.text.split(':')[-2] else int(tag.text.split(':')[-1]) for tag in team_tags]
    bye_points = [int(str(tag.text.split(':')[-1])) if 'BYE' in tag.text.split(':')[-2] else 0 for tag in team_tags]
    div_ids = [did for tag in team_tags]
    team_table = pd.DataFrame({'Team': team_names, 'Team ID': team_ids, 'Match Points': team_scores,'Bye Points': bye_points, 'Div ID': div_ids})
    
    return div_details, team_table

In [3]:
def get_date(timestamp):
    
    # Function to extract the list of match dates from a particular division
    
    lst_dates = timestamp.split(',')
    return [stamp[12:22] for stamp in lst_dates]

In [4]:
def get_results(tags, did, date):
    
    # For a webpage corresponding to a single team match, find the results of each individual player matchup
    # Return the results as a dataframe
    # Subscripts A and B refer to players on left side and right side of the data tables, respectively (NOT the two teams)
    
    discipline, playera_name, playera_team, playera_id, playera_race, playera_wins, playera_bo, playera_snap, playera_RL, playera_score, playerb_name, playerb_team, playerb_id, playerb_race, playerb_wins, playerb_bo, playerb_snap, playerb_RL, playerb_score = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
    # Number of individual matchups is a multiple of 23 (the number of tags corresponding to one match)
    # First two tags correspond to page header
    num_games = (len(tags) - 2)/23
    for i in range(0,num_games):
        if len(tags)>4:
            a_name = str(tags[3 + 23*i].strong.text)
            b_name = str(tags[4 + 23*i].strong.text)
            if 'FORFEIT' not in a_name and 'FORFEIT' not in b_name:
                discipline.append(str(tags[2 +23*i].text))
                playera_name.append(a_name)
                playera_team.append(str(tags[3 + 23*i].small.text).strip('()'))
                playera_id.append(int(str(tags[3 + 23*i].a).split('ID')[1][1:9]))
                playera_race.append(int(tags[6 + 23*i].text))
                playera_wins.append(int(tags[9 + 23*i].text))
                playera_bo.append(tags[12 + 23*i].text)
                playera_snap.append(tags[15 + 23*i].text)
                playera_RL.append(tags[18 + 23*i].text)
                playera_score.append(int(tags[21 + 23*i].strong.text))
                playerb_name.append(b_name)
                playerb_team.append(str(tags[4 + 23*i].small.text).strip('()'))
                playerb_id.append(int(str(tags[4 + 23*i].a).split('ID')[1][1:9]))
                playerb_race.append(int(tags[7 + 23*i].text)) 
                playerb_wins.append(int(tags[10 + 23*i].text))
                playerb_bo.append(tags[13 + 23*i].text)
                playerb_snap.append(tags[16 + 23*i].text)
                playerb_RL.append(tags[19 + 23*i].text)
                playerb_score.append(int(tags[22 + 23*i].strong.text))
    return pd.DataFrame({'Division': did, 'Date': date, 'Discipline': discipline, 'A Name': playera_name, 'A Team': playera_team, 'A ID': playera_id,
                        'A Race': playera_race, 'A Wins': playera_wins, 'A BO': playera_bo, 'A Snap': playera_snap, 'A RL': playera_RL, 
                         'A Score': playera_score, 'B Name': playerb_name,'B Team': playerb_team, 'B ID': playerb_id, 
                         'B Race': playerb_race, 'B Wins': playerb_wins, 'B BO': playerb_bo, 'B Snap': playerb_snap,
                         'B RL': playerb_RL, 'B Score': playerb_score})

In [3]:
# Division ID at which to begin
did1 = 4000
# Number of divisions to scrape
n = 740
divisions = pd.DataFrame()
team_tables = range(0,n)

for i in range(0,n):
    
    # Open each webpage corresponding to a division, collect the division details, and collect the table of competitors
    # Divisions is a dataframe, wheras team_tables is a list of dataframes
    
    did = did1+i
    add = 'https://www.napaleagues.com/states.php?did=' +str(did)
    r = urllib.urlopen(add).read()
    soup = BeautifulSoup(r, 'lxml')
    tags = soup.find_all('td')
    div_details, team_table = get_division(tags, did)
    divisions[i] = div_details
    team_tables[i] = team_table
    
divisions = divisions.T
divisions.columns = ['Division Name', 'Division ID','Location','Weeks','Num Competitors','Format','H2H Bonus','110 Rule','Forfeit Saviour','Dates']

# Combine all team tables into a single dataframe, which is exported to csv
# Export the divisions dataframe to a separate csv 
team_tables_all = pd.concat(team_tables)
add1 = r'C:\\Users\\Owner\\Napa\\divisions' + str(did1) + '_' + str(n) +'.csv'
add2 = r'C:\\Users\\Owner\\Napa\\team_table' + str(did1) + '_' + str(n) +'.csv'
export_divs=divisions.to_csv(add1)
export_teams=team_tables_all.to_csv(add2)
team_tables_all.tail()

,Bye Points,Div ID,Match Points,Team,Team ID
13,0.0,4738.0,108.0,Devon Moreno,32101
14,0.0,4738.0,62.0,Jose Hernandez,32105
15,0.0,4738.0,60.0,Amanda Tebay,32103
16,0.0,4738.0,46.0,Larry Milam jr,32108
17,0.0,4738.0,44.0,Jake Oro,32114


In [186]:
divisions.tail()

,Division Name,Division ID,Location,Weeks,Num Competitors,Format,H2H Bonus,110 Rule,Forfeit Saviour,Dates
5,"Tuesday ""Hampden County"" Standard Limit 8-ball...",3145,"Hampden County, Massachusetts",14,7,5-man Teams,0 points,Yes,No,"[2015-01-20 00:00:00, 2015-01-27 00:00:00, 201..."
6,"Thursday ""RAB"" Lagger's Choice League",3146,"Conway, Arkansas",20,6,5-man Teams,0 points,Yes,No,"[2015-01-22 00:00:00, 2015-01-29 00:00:00, 201..."
7,Tuesday Standard Limit 8-ball League,3147,"LaSalle, Illinois",15,8,4-man Teams,0 points,Yes,No,"[2015-01-20 00:00:00, 2015-01-27 00:00:00, 201..."
8,Monday No Limit 8-ball League,3148,"Folsom, Louisiana",15,5,3-man Teams,0 points,Yes,No,"[2015-01-19 00:00:00, 2015-01-26 00:00:00, 201..."
9,"Monday ""Bankshots In-House"" Lagger's Choice Le...",3149,"Westerville, Ohio",15,5,4-man Teams,0 points,Yes,No,"[2015-01-05 00:00:00, 2015-01-12 00:00:00, 201..."


In [22]:
# The URL for an individual match is constructed using a division ID (did), a team ID (tid), and the date on which the match was played
# For each division, extract the dates from the divisions file, and team IDs from the team_tables file 

divs = pd.read_csv('C:\\Users\\Owner\\Napa\\Divisions\\divisions3000_4740.csv', parse_dates = True)
team_tables = pd.read_csv('C:\\Users\\Owner\\Napa\\Team_tables\\Team_table3100_200.csv')
did1 = 3090
num_divs = 10
divs = divs.dropna()
divs = divs.reset_index()
results_table = []

divs.Dates = divs.apply(lambda x: get_date(x['Dates']),axis=1)

# Only consider 8 ball divisions at this time
divs_sel = divs[divs['Division Name'].str.contains('8')][90:num_divs+90]
dids = [divs_sel['Division ID'].iloc[i] for i in range(0,num_divs)]

for did in dids:
    # Collect dates and teams involved with this division
    dates = divs_sel[divs_sel['Division ID']==did]['Dates'].iloc[0]
    teams = [str(i) for i in team_tables[team_tables['Div ID']==did]['Team ID']]
    for date in dates:
        for tid in teams:
            # Open the webpages for each ma
            add = 'https://www.napaleagues.com/scores.php?tid=' + tid + '&did=' + str(did) + '&date=' + date + '&homeTeam=' +tid +'&format=Team&mid=0&hyper=N'
            r = urllib.urlopen(add).read()
            soup = BeautifulSoup(r, 'lxml')
            tags = soup.find_all('td')
            results_table.append(get_results(tags,did,date))
results_all = pd.concat(results_table)
results_all = results_all.drop_duplicates()
export_results = results_all.to_csv(r'C:\\Users\\Owner\\Napa\\results' + str(did1) +'_' + str(num_divs) + '.csv')
results_all.head()

,A BO,A ID,A Name,A RL,A Race,A Score,A Snap,A Team,A Wins,B BO,...,B Name,B RL,B Race,B Score,B Snap,B Team,B Wins,Date,Discipline,Division
0,-,10006274.0,Becky Watts,-,6.0,15.0,1,The odd couples,6.0,-,...,James Enoch,-,4.0,3.0,-,Wednesday night fun,3.0,2015-02-04,8-ball,3238
1,-,10014524.0,Roger Shepard,-,4.0,14.0,-,Wednesday night fun,4.0,-,...,Hope Meeks,-,3.0,3.0,-,The odd couples,1.0,2015-02-04,8-ball,3238
2,-,10034226.0,Tammy Wilson,-,4.0,3.0,-,The odd couples,2.0,-,...,David Blankenship,-,4.0,14.0,-,Wednesday night fun,4.0,2015-02-04,8-ball,3238
0,-,10004254.0,"Charles Murphy, Sr.",Y,5.0,21.0,-,Murphy's law,5.0,-,...,Danny Obrien,-,3.0,1.0,-,Harley davidson and the marlboro man,0.0,2015-02-04,8-ball,3238
1,-,10004263.0,Tom Winfrey,-,3.0,1.0,-,Murphy's law,0.0,-,...,Gary Hudson,-,3.0,20.0,-,Harley davidson and the marlboro man,3.0,2015-02-04,8-ball,3238
